In [66]:
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
import pandas as pd
import re
import os
import csv
import time
import json
import ast
import pprint
from datetime import datetime, timedelta

In [79]:
def create_geojson(date):
    df_countries_by_day = df_merged[(df_merged.date==date)]
    df_data = df_countries_by_day.groupby(['country','country_2']).agg({'conf_count':'sum', 'cured_count':'sum', 'dead_count':'sum'}).reset_index()
    df_data = df_data.set_index('country')
    
                                          
    json_df = pd.read_json('static/world_geojsons/world.json', encoding='UTF-8')
    json_df_feat = pd.DataFrame(json_df.features)

    geo_dict = {}
    geo_string = ""

    for index, row in json_df_feat.iterrows():
#         print(row['features']['properties']['ADMIN'])
        try:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['ADMIN']
#             print(prop_name)

            prop_american_name = df_data.loc[prop_name,"country_2"]
#             print("AM_NAME", prop_american_name)
            prop_confirmedCount = df_data.loc[prop_name,"conf_count"]
#             print("Conf_C", prop_confirmedCount)
            prop_suspectedCount = 0
            prop_curedCount = df_data.loc[prop_name,"cured_count"]
#             print("Cured_C", prop_curedCount)
            prop_deadCount = df_data.loc[prop_name,"dead_count"]    
#             print("Dead_C", prop_deadCount)
            prop_date = date
#             print("Date:", prop_date)

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
#             print(str_prop_all)
        #     print('=============================')    
        #     print(row['features']['geometry'])
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
        #     print(str_geom_1)
        #     print('=============================')

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
        #     print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
        except:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['ADMIN']

            prop_american_name = ""
            prop_confirmedCount = 0
            prop_suspectedCount = 0
            prop_curedCount = 0
            prop_deadCount = 0  
            prop_date = date

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
        #     print(str_prop_all)
        #     print('=============================')    
        #     print(row['features']['geometry'])
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
        #     print(str_geom_1)
        #     print('=============================')

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
        #     print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
    pre_fix = "{\"type\": \"FeatureCollection\", \"features\": ["
    post_fix = "]}"

    total_string = pre_fix + geo_string[:-1] + post_fix

#     print(total_string)
    output_path = os.path.join("static/world_geojsons", date + ".json")
    with open(output_path, "w", encoding='UTF-8') as text_file:
        text_file.write(total_string)
        text_file.close()

In [3]:
# Get the data from 3 different data source
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_confirmed = pd.read_csv(confirmed_url)
cured_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df_cured = pd.read_csv(cured_url)
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
df_death = pd.read_csv(death_url)
# total_columns = len(df_death.columns)
# df_death.head()
# tables.describe()
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,982,986,987,988,989,989,989,989,989,989
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,387,393,395,396,399,399,399,400,400,410
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,555,560,567,572,573,575,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,292,293,293,293,293,293,293,294,294,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,91


In [4]:
# Create unique rows by moving date column to row level for all Confirmed cases
df_confirmed_long = df_confirmed.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_confirmed_long.head()
output_path = os.path.join("static/world_data", "df_world_confirmed_original.csv")
df_confirmed.to_csv(output_path)

# Create unique rows by moving date column to row level for all Cured cases
df_cured_long = df_cured.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_cured_long.head()
output_path = os.path.join("static/world_data", "df_world_cured_original.csv")
df_cured.to_csv(output_path)

# Create unique rows by moving date column to row level for all Death cases
df_death_long = df_death.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_death_long.head()
output_path = os.path.join("static/world_data", "df_world_death_original.csv")
df_death.to_csv(output_path)

df_confirmed_long.head()
df_confirmed_long["new_date"] = pd.to_datetime(df_confirmed_long['Date'])
# df_confirmed_long.drop(['Date'], axis=1)

# df_temp=df_death_long[df_death_long["Province/State"].isnull()]
# df_temp.head()
df_confirmed_long.head()

,Province/State,Country/Region,Lat,Long,Date,Value,new_date
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1,2020-01-22
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14,2020-01-22
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6,2020-01-22
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1,2020-01-22
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0,2020-01-22


In [5]:
# If Province/State isn't filled, use the country value instead

df_confirmed_long["Province/State"].fillna(df_confirmed_long["Country/Region"], inplace=True)
df_confirmed_long = df_confirmed_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long", "Date":"date", "Value": "conf_count"})

df_cured_long["Province/State"].fillna(df_cured_long["Country/Region"], inplace=True)
df_cured_long = df_cured_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "cured_count"})

df_death_long["Province/State"].fillna(df_death_long["Country/Region"], inplace=True)
df_death_long = df_death_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "dead_count"})


In [6]:

df_confirmed_long.loc[(df_confirmed_long.country == 'Mainland China'),'country']='China'
df_confirmed_long = df_confirmed_long.set_index(['date', 'american_name'])

df_cured_long.loc[(df_cured_long.country == 'Mainland China'),'country']='China'
df_cured_long = df_cured_long.set_index(['date', 'american_name'])

df_death_long.loc[(df_death_long.country == 'Mainland China'),'country']='China'
df_death_long = df_death_long.set_index(['date', 'american_name'])


In [7]:

df_merged = pd.merge(df_confirmed_long, df_cured_long, how='left', on=['date', 'american_name', 'country','lat','long'])

df_merged = pd.merge(df_merged, df_death_long,how='left', on=['date', 'american_name', 'country','lat','long'])
df_merged = df_merged.reset_index()
df_merged = df_merged.drop(['date'], axis=1)
df_merged['country_2'] = df_merged['country']
df_merged = df_merged.rename(columns={"new_date":"date"})

output_path = os.path.join("static/world_data", "df_world_all.csv")
df_merged.to_csv(output_path)


df_merged.head()

,american_name,country,lat,long,conf_count,date,cured_count,dead_count,country_2
0,Anhui,China,31.8257,117.2264,1,2020-01-22,0,0,China
1,Beijing,China,40.1824,116.4142,14,2020-01-22,0,0,China
2,Chongqing,China,30.0572,107.8740,6,2020-01-22,0,0,China
3,Fujian,China,26.0789,117.9874,1,2020-01-22,0,0,China
4,Gansu,China,36.0611,103.8343,0,2020-01-22,0,0,China


In [80]:
current_date = datetime.now()
print("Current Date ",current_date)
start_date = datetime.strptime('2020-01-26', "%Y-%m-%d")
print("Start Date", start_date)
# end_date = start_date + timedelta(days=10)
# print(end_date)

while current_date >= start_date:
    pass_date = start_date.strftime("%Y-%m-%d")
    print(pass_date)
    create_geojson(pass_date)
    start_date = start_date + timedelta(days=1)

# create_geojson('2020-02-26')

Current Date  2020-02-27 20:15:20.515183
Start Date 2020-01-26 00:00:00
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
